In [1]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [34]:
!git clone https://github.com/ZXWBOT/nose.git

Cloning into 'nose'...
remote: Enumerating objects: 117, done.
remote: Total 117 (delta 0), reused 0 (delta 0), pack-reused 117
Receiving objects: 100% (117/117), 42.47 MiB | 23.22 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
!python /content/drive/MyDrive/nose/xml_to_csv.py

Successfully converted xml to csv.


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/nose/road_signs_labels.csv')
df.sort_values(by=['filename'], axis=0)
df.head(2)

,filename,width,height,class,xmin,ymin,xmax,ymax
0,22.png,586,553,nose,218,230,370,362
1,21.png,651,664,nose,211,342,365,449


In [ ]:
!python /content/drive/MyDrive/nose/convert_csv_to_coco.py

In [ ]:
import json
import os
import random


def split_dataset(input_json, val_ratio, random_seed):
    random.seed(random_seed)

    with open(input_json) as json_reader:
        dataset = json.load(json_reader)

    images = dataset['images']
    annotations = dataset['annotations']
    categories = dataset['categories']

    # file_name에 prefix 디렉토리까지 포함 (CocoDataset 클래스를 사용하는 경우)
    for image in images:
        image['file_name'] = '{}'.format(image['file_name'])

    image_ids = [x.get('id') for x in images]
    image_ids.sort()
    random.shuffle(image_ids)

    num_val = int(len(image_ids) * val_ratio)
    num_train = len(image_ids) - num_val

    image_ids_val, image_ids_train = set(image_ids[:num_val]), set(image_ids[num_val:])

    train_images = [x for x in images if x.get('id') in image_ids_train]
    val_images = [x for x in images if x.get('id') in image_ids_val]
    train_annotations = [x for x in annotations if x.get('image_id') in image_ids_train]
    val_annotations = [x for x in annotations if x.get('image_id') in image_ids_val]

    train_data = {
        'images': train_images,
        'annotations': train_annotations,
        'categories': categories,
    }

    val_data = {
        'images': val_images,
        'annotations': val_annotations,
        'categories': categories,
    }

In [ ]:
split_dataset(input_json='/content/drive/MyDrive/DogNose/mmdetection/nose/annotation_coco.json',
              val_ratio=0.1,
              random_seed=2023)

In [35]:
!pwd

/content/drive/MyDrive


In [36]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 15978, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 15978 (delta 89), reused 112 (delta 70), pack-reused 15831
Receiving objects: 100% (15978/15978), 14.60 MiB | 23.01 MiB/s, done.
Resolving deltas: 100% (10962/10962), done.


In [5]:
%cd /content/drive/MyDrive/yolov5/
!pip install -r requirements.txt

/content/drive/MyDrive/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.4/595.4 kB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00


In [44]:
%cd /
from glob import glob
img_list = glob('/content/drive/MyDrive/dog_nose/images/*.png')
print(len(img_list))

/
50


In [47]:
from sklearn.model_selection import train_test_split
train_img_list, val_img_list = train_test_split(img_list, test_size = 0.1, random_state = 2000)
print(len(train_img_list), len(val_img_list))

45 5


In [49]:
with open('/content/drive/MyDrive/dog_nose/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/drive/MyDrive/dog_nose/dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [53]:
import yaml

with open('/content/drive/MyDrive/dog_nose/dataset/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/content/drive/MyDrive/dog_nose/dataset/train.txt'
data['val'] = '/content/drive/MyDrive/dog_nose/dataset/val.txt'

with open('/content/drive/MyDrive/dog_nose/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'names': ['nose'], 'nc': 1, 'train': 'dog_nose/train/images', 'val': 'dog_nose/valid/images'}
{'names': ['nose'], 'nc': 1, 'train': '/content/drive/MyDrive/dog_nose/dataset/train.txt', 'val': '/content/drive/MyDrive/dog_nose/dataset/val.txt'}


In [56]:
import os
import glob
import xml.etree.ElementTree as ET

# xml 파일이 위치한 디렉토리 경로
xml_dir_path = '/content/drive/MyDrive/nose/annotations'

# 결과 파일이 저장될 디렉토리 경로
output_dir_path = '/content/drive/MyDrive/dog_nose/labels'

# xml 파일들을 모두 읽습니다.
xml_files = glob.glob(os.path.join(xml_dir_path, '*.xml'))

for xml_file in xml_files:
    # xml 파일을 파싱합니다.
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # 이미지의 크기 정보를 가져옵니다.
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    # 객체 정보를 가져옵니다.
    for obj in root.findall('object'):
        # bounding box 정보를 가져옵니다.
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # YOLO 형식으로 변환합니다.
        x_center = (xmin + xmax) / (2 * width)
        y_center = (ymin + ymax) / (2 * height)
        w = (xmax - xmin) / width
        h = (ymax - ymin) / height

        # class id를 설정합니다.
        # 여기서는 'nose' 클래스가 0이라고 가정합니다.
        class_id = 0

        # 결과를 출력 파일에 저장합니다.
        filename = os.path.basename(xml_file)
        filename_without_ext = os.path.splitext(filename)[0]
        txt_file_path = os.path.join(output_dir_path, filename_without_ext + '.txt')
        with open(txt_file_path, 'a') as f:
            f.write(f'{class_id} {x_center} {y_center} {w} {h}\n')


In [65]:
%cd /content/drive/MyDrive/yolov5

!python train.py --img 416 --batch 32 --epochs 100 --data /content/drive/MyDrive/dog_nose/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name results

/content/drive/MyDrive/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/drive/MyDrive/dog_nose/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-178-ga199480 Python-3.10.11 torch-1.12.0+cu113 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5,

In [3]:
%cd /content/drive/MyDrive/yolov5

/content/drive/MyDrive/yolov5


In [17]:
!python detect.py --weights /content/drive/MyDrive/yolov5/runs/train/results3/weights/best.pt --source /content/drive/MyDrive/dog_nose/test_images/dog6_2.jpg

detect: weights=['/content/drive/MyDrive/yolov5/runs/train/results3/weights/best.pt'], source=/content/drive/MyDrive/dog_nose/test_images/dog6_2.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-178-ga199480 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/drive/MyDrive/dog_nose/test_images/dog6_2.jpg: 640x480 1 nose, 39.5ms
Speed: 0.6ms pre-process, 39.5ms inference, 1.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp20
